In [71]:
import numpy as np
import torch
import sys
from pathlib import Path
import importlib

import pandas as pd
import torchaudio

# module_path = str(Path.cwd().parents[0] / "utils")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if module_path not in sys.path:
#     sys.path.append(module_path)
# module_path = str(Path.cwd().parents[0] / "network_models/w2v_emotion")
# if module_path not in sys.path:
#     sys.path.append(module_path)
# module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
# if module_path not in sys.path:
#     sys.path.append(module_path)

import utils.wav2Vec_utils as w2vU
import utils.audio_dataset_utils as ADU
import network_models.soundstream_lstm.LSTM_dataset as lds
import network_models.soundstream_lstm.CombinedEmoDataset_7_Emo as ced
import  network_models.w2v_emotion_model.custom_collator as cc
import network_models.w2v_emotion_model.custom_model as cm
import network_models.w2v_emotion_model.model_trainer as ct
import gc

model_name_or_path = "facebook/wav2vec2-large-960h-lv60-self"
pooling_mode = "mean"
device = "cuda"
emo_dataset = ced.CombinedEmoDataSet_7_emos(directory_tess="/home/ckwdani/Music/emotionDatasets/converted_mono/tess",
                                            directory_cafe="/home/ckwdani/Music/emotionDatasets/converted_mono/cafe",
                                            directory_ravdess="/home/ckwdani/Music/emotionDatasets/converted_mono/RAVDESS Audio_Speech_Actors_01-24",
                                            directory_mesd="/home/ckwdani/Music/emotionDatasets/converted_mono/mesd",
                                            with_dataset=True,
                                            transFormAudio=ced.collateToSeconds(3.5, 16000, const_value=0, device=device),
                                            device=device)

inputcolumn = "encoded"
labelcolumn = "emotionCode"
dataset_column = "dataset"
clear_label_colums = "clear_emotion"


#emo_dataset = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess", device=device)
def encodewithWav2Vec(model, dataSet, cf):

    def emoToId(emotion: str):
        return np.where(emo_dataset.label_list == emotion)[0]

    tensors = []
    emotions = []
    emotions_names = []
    dataset_names = []
    i = 0
    for sample in iter(dataSet):
        i += 1
        from utils.Visual_Coding_utils import progress
        progress(i, dataSet.__len__(), "generating encoding")
        with torch.no_grad():
            t = torch.tensor(sample[0])
            t = torch.unsqueeze(t, dim=0).to(device)
            data = model(t)




        tensors.append(data[0].detach().cpu())
        emotions.append(emoToId(sample[1]))
        emotions_names.append(sample[1])

    itersize = 2500
    df = pd.DataFrame()
    dfInter = pd.DataFrame()

    parts = len(tensors)//itersize
    end = len(tensors)%itersize

    df[inputcolumn] = tensors[0:itersize]
    df[labelcolumn] = emotions[0:itersize]
    df[clear_label_colums] = emotions_names[0:itersize]
    for i in range(parts - 1):
        dfInter = pd.DataFrame()
        dfInter[inputcolumn] = tensors[(i+1)*itersize:(i+2)*itersize]
        dfInter[labelcolumn] = emotions[(i+1)*itersize:(i+2)*itersize]
        dfInter[clear_label_colums] = emotions_names[(i+1)*itersize:(i+2)*itersize]
        gc.collect()
        df = df.append(dfInter)

    dfInter = pd.DataFrame()
    gc.collect()
    if itersize < len(tensors) and end != 0:
        dfInter[inputcolumn] = tensors[(parts*itersize):(parts*itersize+end)]
        dfInter[labelcolumn] = emotions[(parts*itersize):(parts*itersize+end)]
        dfInter[clear_label_colums] = emotions_names[(parts*itersize):(parts*itersize+end)]
        gc.collect()
        df = df.append(dfInter)
    gc.collect()



    #df[self.labelcolumn] = emotions
    return df


In [73]:
processor, sr = w2vU.init_w2v2(num_labels=len(emo_dataset.label_list), label_list=emo_dataset.label_list, device=device)
newSet = lds.AudioEmotionTessWav2VecDataset(emo_dataset, processor= processor, sampling_rate=sr)
gc.collect()
modelEncodeModel = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device, num_emotions=len(emo_dataset.label_list), dataset_generator=True)
modelEncodeModel.freeze_feature_extractor()

In [74]:

data_collator = cc.DataCollatorCTCWithPadding(processor=processor, padding=True, num_labels=len(newSet.dataSet.label_list))

data = encodewithWav2Vec(modelEncodeModel, dataSet=newSet, cf = data_collator.collate_fn)


# trainDS, testDs = ADU.train_val_dataset(newSet, val_split=0.2, seed=100)
# valDs, testDs = ADU.train_val_dataset(testDs, val_split=0.5, seed=100)
#trainSet, evalSet = ADU.train_val_dataset(newSet, 0.1)
#trainSet, evalSet = ADU.train_val_dataset(evalSet, 0.1)

#model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device, num_emotions=len(emo_dataset.label_list), from_full_dataset=True)



# trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=2, save_model_every=500, num_epochs=5001, model_path="../content/classifier/W2VClassifier/Nr1/", data_collator=data_collator)
# gc.collect()
# trainer.train()

/tmp/ipykernel_69955/2431360585.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dfInter)
/tmp/ipykernel_69955/2431360585.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dfInter)


In [78]:
trainDS, testDs = ADU.train_val_dataset(data, val_split=0.2, seed=100)
valDs, testDs = ADU.train_val_dataset(testDs, val_split=0.5, seed=100)

In [77]:
Path("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks_clip/content/datasets/w2vEnc").mkdir(parents=True)
data.to_pickle("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks_clip/content/datasets/w2vEnc/w2v_3_5.pkl")
model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device, num_emotions=len(emo_dataset.label_list), from_full_dataset=True)

In [88]:
importlib.reload(ct)
trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainDS, eval_dataset=valDs, device=device, batch_size=6, save_model_every=100, num_epochs=1001, model_path="../content/classifier/W2VClassifier/final/", data_collator=data_collator, labelList=emo_dataset.label_list)
gc.collect()
trainer.train()

Epoch 1
-------------------------------


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 408

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/venv/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
  

In [83]:
from network_models.soundsream_models_and_utils.ss_trainer_gen_models import SSGenModelTrainer

batch_size = 4
models_dir = "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/classifier/W2VClassifier/final/"
#models_dir = "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/3_5_sec_dimred/Nr1/"
trials_per_model_type = 2
epochs = 1000
save_highest_acc_min_acc = 0.5
save_model_every = 50
lr = 1e-4
lr_quotient = 2

trainer = SSGenModelTrainer(lr=lr, num_epochs=epochs, model=model, train_dataset=trainDS,
                            eval_dataset=valDs,
                            device=device, labelList=emo_dataset.label_list,
                            batch_size=batch_size,
                            save_model_every=save_model_every,
                            save_highest_acc_min_acc=save_highest_acc_min_acc,
                            model_path = models_dir)

In [87]:
trainDS

/home/ckwdani/.local/share/JetBrains/Toolbox/apps/PyCharm-P/ch-0/222.4345.23/plugins/python/helpers/pydev/_pydevd_bundle/pydevd_utils.py:605: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  s = s[:max_items]
/home/ckwdani/.local/share/JetBrains/Toolbox/apps/PyCharm-P/ch-0/222.4345.23/plugins/python/helpers/pydev/_pydevd_bundle/pydevd_utils.py:606: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in s.iteritems():
/home/ckwdani/.local/share/JetBrains/Toolbox/apps/PyCharm-P/ch-0/222.4345.23/plugins/python/helpers/pydev/_pydevd_bundle/pydevd_utils.py:605: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be t

KeyboardInterrupt: 

In [17]:
l[0].shape

torch.Size([15884])

In [2]:
#model.load_state_dict(torch.load("../content/classifier/W2VClassifier/Nr1/emo_reco_950.pth"))
#model.from_pretrained()
#model(torch.unsqueeze(torch.tensor(trainSet[0][0]).cuda(), dim=0))